In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import random

import torch
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

현재 CUDA가 사용 가능한지 또한 어떤 device가 사용 가능한지를 확인하고
특정 device를 사용하도록 설정할 수 있다.

Tensor 생성 후 해당 tensor를 GPU에 loading해주기!

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [4]:
train = pd.read_csv("../input/2021-ai-w5-p1/train.csv")
test = pd.read_csv("../input/2021-ai-w5-p1/test.csv")

In [5]:
train.head()

# 시각화

In [6]:
r = random.randint(0, len(train) - 1)
X_single_data = np.array(train.iloc[r,:-1])

import matplotlib.pyplot as plt
plt.imshow(X_single_data.reshape(28,28))
print(train.iloc[r,0])

* feature target 분리
* 0~255 로 정규화

In [7]:
# 0~255
x_train = train.drop(["label"], axis=1)
x_train /= 255
x_test = test / 255
y_train = train.label

In [8]:
x_train_data=torch.FloatTensor(np.array(x_train)).to(device)
y_train_data=torch.LongTensor(np.array(y_train)).to(device)
x_test_data=torch.FloatTensor(np.array(x_test)).to(device)

print(x_train_data.shape, y_train_data.shape)
print(x_test_data.shape)

# 파라미터 설정 - 학습률 & epoch

In [9]:
learning_rate = 1e-2
training_epochs = 3000

# 모델 학습
* F.cross_entropy는 비용 함수에 소프트맥스 함수까지 포함하고 있음!

In [10]:
features = 28 * 28
nbclass = 10

W = torch.zeros((features, nbclass), requires_grad=True, device = device) #기울기
b = torch.zeros(1, requires_grad=True, device = device) #y절편

optimizer = optim.SGD([W, b], lr = learning_rate)

for epoch in range(training_epochs + 1):
    
    cost = F.cross_entropy((x_train_data.matmul(W) + b), y_train_data) #비용
    optimizer.zero_grad()
    #pytorch에서는 gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문"에 
    #우리는 항상 backpropagation을 하기전에 gradients를 zero로 만들어주고 시작을 해야합니다.
    cost.backward() #역전파 => 가중치, 편향값 보정
    optimizer.step()
    

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, training_epochs, cost.item()))

print('Learning finished')


# Train 정확도 평가
* torch.no_grad() 함수는 gradient를 계산해주는 context를 비활성화시켜 필요한 메모리를 줄이고 연산 속도를 늘림!!

In [11]:
with torch.no_grad():
    hypothesis = F.softmax(x_train_data.matmul(W) + b, dim=1) #0과 1사이의 값을 가지는 벡터로 변환
    predict = torch.argmax(hypothesis, dim=1)

    correct_prediction = predict.float() == y_train_data
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

# Test 데이터 예측

In [12]:
with torch.no_grad():
    hypothesis = F.softmax(x_test_data.matmul(W) + b, dim=1)
    predict = torch.argmax(hypothesis, dim=1)

In [14]:
submit = pd.read_csv("../input/2021-ai-w5-p1/sample_submit.csv")

for i, value in enumerate(predict):
    submit["label"][i] =value.item()

submit.label = submit.label.astype(int)
    
submit.to_csv("kiy-w5p1.csv", index = False)

# 참고자료
* pytorch 문법
https://wikidocs.net/60572
* python with문 이해
https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=wideeyed&logNo=221653260516